In [ ]:
!pip install implicit

import pandas as pd
import numpy as np
import scipy.sparse as sparse
import implicit
from catboost import CatBoostRanker, Pool
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
import warnings
from pathlib import Path
from typing import List, Dict, Tuple, Optional, Union
import random
import gc
import os
from tqdm import tqdm

# Настройки
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

# ============ КОНСТАНТЫ И КОНФИГУРАЦИЯ ============

class Constants:
    """Класс с константами для имен колонок и значений"""
    # Имена файлов
    TRAIN_FILE = "train.csv"
    CANDIDATES_FILE = "candidates.csv"
    USERS_FILE = "users.csv"
    BOOKS_FILE = "books.csv"
    GENRES_FILE = "genres.csv"
    BOOK_GENRES_FILE = "book_genres.csv"
    BOOK_DESCRIPTIONS_FILE = "book_descriptions.csv"
    
    # Имена колонок
    COL_USER_ID = "user_id"
    COL_BOOK_ID = "book_id"
    COL_TIMESTAMP = "timestamp"
    COL_HAS_READ = "has_read"
    COL_RATING = "rating"
    COL_TARGET = "target"
    
    # Метаданные пользователей
    COL_GENDER = "gender"
    COL_AGE = "age"
    
    # Метаданные книг
    COL_AUTHOR_ID = "author_id"
    COL_PUBLISHER = "publisher"
    COL_LANGUAGE = "language"
    COL_TITLE = "title"
    COL_AUTHOR_NAME = "author_name"
    COL_PUBLICATION_YEAR = "publication_year"
    COL_IMAGE_URL = "image_url"
    COL_DESCRIPTION = "description"
    COL_GENRE_ID = "genre_id"
    COL_GENRE_NAME = "genre_name"
    
    # Значения для целевой переменной
    TARGET_READ = 2      # Прочитал
    TARGET_PLANNED = 1   # Планирует прочитать
    TARGET_NEGATIVE = 0  # Негативный пример


class Config:
    """Класс конфигурации с параметрами проекта"""
    # --- ПУТИ ---
    ROOT_DIR = Path(".")
    DATA_DIR = ROOT_DIR / "/kaggle/input/nto-team-tour" / "public" # ЗАМЕНИТЕ НА СВОЮ ССЫЛКУ
    MODEL_DIR = ROOT_DIR / "models"
    RESULTS_DIR = ROOT_DIR / "results"
    
    # Создание директорий
    MODEL_DIR.mkdir(parents=True, exist_ok=True)
    RESULTS_DIR.mkdir(parents=True, exist_ok=True)
    
    # --- ПАРАМЕТРЫ ---
    RANDOM_STATE = 42
    VAL_SPLIT_SIZE = 0.1  # Размер валидационной выборки
    TEMPORAL_VAL_DAYS = 30  # Дней для временного сплита
    
    # --- ALS ПАРАМЕТРЫ ---
    ALS_FACTORS = 64
    ALS_REGULARIZATION = 0.05
    ALS_ITERATIONS = 40
    ALS_TOP_K_CANDIDATES = 50
    
    # Веса взаимодействий для ALS
    ALS_WEIGHT_READ = 4      # Прочитал
    ALS_WEIGHT_PLANNED = 1   # Планирует
    
    # --- CatBoost ПАРАМЕТРЫ ---
    CB_ITERATIONS = 5000
    CB_LEARNING_RATE = 0.02
    CB_DEPTH = 10
    CB_L2_LEAF_REG = 10
    CB_EVAL_METRIC = "NDCG:top=20"
    CB_EARLY_STOPPING_ROUNDS = 150
    
    # --- TF-IDF ПАРАМЕТРЫ ---
    TFIDF_MAX_FEATURES = 1000
    TFIDF_SVD_COMPONENTS = 400
    
    # --- ФИЧИ ---
    CATEGORICAL_FEATURES = [
        Constants.COL_GENDER,
        Constants.COL_AUTHOR_ID,
        Constants.COL_PUBLISHER,
        Constants.COL_LANGUAGE
    ]
    
    TEXT_FEATURES_TO_DROP = [
        Constants.COL_TITLE,
        Constants.COL_AUTHOR_NAME,
        Constants.COL_DESCRIPTION,
        Constants.COL_GENRE_NAME
    ]
    
    @classmethod
    def get_cb_params(cls):
        """Возвращает параметры для CatBoost"""
        return {
            'loss_function': 'YetiRank',
            'iterations': cls.CB_ITERATIONS,
            'learning_rate': cls.CB_LEARNING_RATE,
            'depth': cls.CB_DEPTH,
            'l2_leaf_reg': cls.CB_L2_LEAF_REG,
            'random_seed': cls.RANDOM_STATE,
            'eval_metric': cls.CB_EVAL_METRIC,
            'verbose': 100,
            'early_stopping_rounds': cls.CB_EARLY_STOPPING_ROUNDS,
            'task_type':"GPU"
        }


def seed_everything(seed: int = 42):
    """Установка сидов для воспроизводимости"""
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)


# ============ УТИЛИТЫ ============

class DataOptimizer:
    """Класс для оптимизации памяти в данных"""
    
    @staticmethod
    def reduce_mem_usage(df: pd.DataFrame) -> pd.DataFrame:
        """
        Итеративно изменяет типы данных для уменьшения использования памяти
        
        Args:
            df: DataFrame для оптимизации
            
        Returns:
            Оптимизированный DataFrame
        """
        start_mem = df.memory_usage().sum() / 1024**2
        print(f"Начальное использование памяти: {start_mem:.2f} MB")
        
        for col in df.columns:
            col_type = df[col].dtype
            
            # Пропускаем нечисловые типы и даты
            if (col_type != object and 
                not str(col_type).startswith('datetime') and
                col_type.name != 'category'):
                
                c_min = df[col].min()
                c_max = df[col].max()
                
                # Оптимизация целых чисел
                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                # Оптимизация чисел с плавающей точкой
                else:
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
        
        end_mem = df.memory_usage().sum() / 1024**2
        reduction = 100 * (start_mem - end_mem) / start_mem
        print(f"Конечное использование памяти: {end_mem:.2f} MB")
        print(f"Сокращение на {reduction:.1f}%")
        
        return df
    
    @staticmethod
    def load_and_optimize(filepath: Path, **kwargs) -> pd.DataFrame:
        """
        Загружает и оптимизирует DataFrame
        
        Args:
            filepath: Путь к файлу
            **kwargs: Дополнительные аргументы для pd.read_csv
            
        Returns:
            Оптимизированный DataFrame
        """
        print(f"Загрузка {filepath.name}...")
        df = pd.read_csv(filepath, **kwargs)
        df = DataOptimizer.reduce_mem_usage(df)
        return df


# ============ ЗАГРУЗКА ДАННЫХ ============

class DataLoader:
    """Класс для загрузки и подготовки данных"""
    
    def __init__(self, config: Config):
        self.config = config
        self.constants = Constants()
    
    def load_all_data(self) -> Dict[str, pd.DataFrame]:
        """
        Загружает все необходимые данные
        
        Returns:
            Словарь с DataFrame
        """
        data = {}
        
        # Загрузка тренировочных данных
        data['train'] = DataOptimizer.load_and_optimize(
            self.config.DATA_DIR / self.constants.TRAIN_FILE,
            parse_dates=[self.constants.COL_TIMESTAMP]
        )
        
        # Загрузка кандидатов
        data['candidates'] = DataOptimizer.load_and_optimize(
            self.config.DATA_DIR / self.constants.CANDIDATES_FILE
        )
        
        # Загрузка метаданных пользователей
        data['users'] = DataOptimizer.load_and_optimize(
            self.config.DATA_DIR / self.constants.USERS_FILE
        )
        
        # Загрузка метаданных книг
        data['books'] = DataOptimizer.load_and_optimize(
            self.config.DATA_DIR / self.constants.BOOKS_FILE
        )
        
        # Загрузка описаний книг
        try:
            data['book_descriptions'] = DataOptimizer.load_and_optimize(
                self.config.DATA_DIR / self.constants.BOOK_DESCRIPTIONS_FILE
            )
            print(f"Book descriptions shape: {data['book_descriptions'].shape}")
        except:
            print("Book descriptions file not found, skipping...")
            data['book_descriptions'] = pd.DataFrame()
        
        # Загрузка жанров
        try:
            data['genres'] = DataOptimizer.load_and_optimize(
                self.config.DATA_DIR / self.constants.GENRES_FILE
            )
            data['book_genres'] = DataOptimizer.load_and_optimize(
                self.config.DATA_DIR / self.constants.BOOK_GENRES_FILE
            )
            print(f"Genres shape: {data['genres'].shape}")
            print(f"Book genres shape: {data['book_genres'].shape}")
        except:
            print("Genres files not found, skipping...")
            data['genres'] = pd.DataFrame()
            data['book_genres'] = pd.DataFrame()
        
        print(f"Train shape: {data['train'].shape}")
        print(f"Candidates shape: {data['candidates'].shape}")
        print(f"Users shape: {data['users'].shape}")
        print(f"Books shape: {data['books'].shape}")
        
        return data

    
    
    def prepare_target(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Подготавливает целевую переменную для ранжирования
        
        Args:
            df: DataFrame с взаимодействиями
            
        Returns:
            DataFrame с подготовленной целевой переменной
        """
        df = df.copy()
        
        # Создаем целевую переменную с градациями
        df[self.constants.COL_TARGET] = df[self.constants.COL_HAS_READ].apply(
            lambda x: self.constants.TARGET_READ if x == 1 
                     else self.constants.TARGET_PLANNED
        )
        
        return df


# ============ ALS МОДЕЛЬ ============

class ALSRecommender:
    """Класс для генерации кандидатов с помощью ALS"""
    
    def __init__(self, config: Config):
        self.config = config
        self.model = None
        self.user2idx = None
        self.item2idx = None
        self.idx2item = None
        self.sparse_matrix = None
    
    def prepare_sparse_matrix(self, df: pd.DataFrame) -> sparse.csr_matrix:
        """
        Подготавливает разреженную матрицу для ALS
        
        Args:
            df: DataFrame с взаимодействиями
            
        Returns:
            Разреженная матрица взаимодействий
        """
        # Создаем маппинги
        unique_users = df[Constants.COL_USER_ID].unique()
        unique_items = df[Constants.COL_BOOK_ID].unique()
        
        self.user2idx = {u: i for i, u in enumerate(unique_users)}
        self.item2idx = {i: idx for idx, i in enumerate(unique_items)}
        self.idx2item = {idx: i for i, idx in self.item2idx.items()}
        
        # Создаем веса взаимодействий (читал = 4, планирует = 1)
        df['weight'] = df[Constants.COL_HAS_READ].apply(
            lambda x: 4 if x == 1 else 1
        )
        
        # Создаем разреженную матрицу
        rows = df[Constants.COL_USER_ID].map(self.user2idx).values
        cols = df[Constants.COL_BOOK_ID].map(self.item2idx).values
        data = df['weight'].values
        
        sparse_matrix = sparse.csr_matrix(
            (data, (rows, cols)), 
            shape=(len(unique_users), len(unique_items))
        )
        
        return sparse_matrix
    
    def train(self, df: pd.DataFrame):
        """
        Обучает ALS модель
        
        Args:
            df: DataFrame с взаимодействиями
        """
        print("Обучение ALS модели...")
        self.sparse_matrix = self.prepare_sparse_matrix(df)
        
        self.model = implicit.als.AlternatingLeastSquares(
            factors=self.config.ALS_FACTORS,
            regularization=self.config.ALS_REGULARIZATION,
            iterations=self.config.ALS_ITERATIONS,
            random_state=self.config.RANDOM_STATE
        )
        
        self.model.fit(self.sparse_matrix)
        print("ALS модель обучена")
    
    def generate_candidates(self, df: pd.DataFrame, users_to_predict: List[int], 
                           n_candidates: int = 50) -> pd.DataFrame:
        """
        Генерирует кандидатов для указанных пользователей.
        Кандидаты, которые есть в реальном df, станут позитивными.
        Те, которых нет - негативными (hard negatives).
        
        Args:
            df: DataFrame с реальными взаимодействиями
            users_to_predict: Список ID пользователей
            n_candidates: Количество кандидатов на пользователя
            
        Returns:
            DataFrame с кандидатами
        """
        if self.model is None:
            raise ValueError("Модель ALS не обучена. Сначала вызовите train()")
        
        # Фильтруем пользователей, которых нет в обучении (cold start)
        valid_users = [u for u in users_to_predict if u in self.user2idx]
        valid_user_idxs = [self.user2idx[u] for u in valid_users]
        
        if not valid_users:
            print("Нет валидных пользователей для рекомендаций")
            return pd.DataFrame(columns=[Constants.COL_USER_ID, Constants.COL_BOOK_ID])
        
        print(f"Генерация {n_candidates} кандидатов для {len(valid_users)} пользователей...")
        
        # Генерация рекомендаций
        # filter_already_liked_items=False важно - хотим видеть и то, что уже лайкнул, чтобы разметить правильно
        ids, scores = self.model.recommend(
            valid_user_idxs,
            self.sparse_matrix[valid_user_idxs],
            N=n_candidates,
            filter_already_liked_items=False
        )
        
        # Собираем кандидатов в DataFrame
        candidates_list = []
        for i, user_id in enumerate(valid_users):
            item_idxs = ids[i]
            for item_idx in item_idxs:
                book_id = self.idx2item[item_idx]
                candidates_list.append([user_id, book_id])
        
        candidates_df = pd.DataFrame(
            candidates_list,
            columns=[Constants.COL_USER_ID, Constants.COL_BOOK_ID]
        )
        
        # Добавляем информацию о том, является ли кандидат реальным взаимодействием
        real_interactions_set = set(
            zip(df[Constants.COL_USER_ID], df[Constants.COL_BOOK_ID])
        )
        
        candidates_df['is_real'] = candidates_df.apply(
            lambda row: (row[Constants.COL_USER_ID], 
                        row[Constants.COL_BOOK_ID]) in real_interactions_set,
            axis=1
        )
        
        print(f"Сгенерировано {len(candidates_df)} кандидатов")
        print(f"  - Реальных взаимодействий: {candidates_df['is_real'].sum()}")
        print(f"  - Негативных кандидатов: {(~candidates_df['is_real']).sum()}")
        
        return candidates_df


# ============ ГЕНЕРАЦИЯ ПРИЗНАКОВ ============

class TFIDFFeatureExtractor:
    """Класс для извлечения TF-IDF признаков из текста"""
    
    def __init__(self, config: Config):
        self.config = config
        self.tfidf_vectorizer = None
        self.svd = None
        
    def fit_transform(self, texts: List[str]) -> np.ndarray:
        """
        Обучает TF-IDF и SVD на текстах
        
        Args:
            texts: Список текстов
            
        Returns:
            Матрица TF-IDF признаков с пониженной размерностью
        """
        print("Обучение TF-IDF и SVD...")
        
        # Заполняем пропуски пустыми строками
        texts = [str(text) if pd.notna(text) else '' for text in texts]
        
        # Создаем и обучаем TF-IDF
        self.tfidf_vectorizer = TfidfVectorizer(
            max_features=self.config.TFIDF_MAX_FEATURES,
            stop_words=None,
            min_df=2,
            max_df=0.95
        )
        
        tfidf_matrix = self.tfidf_vectorizer.fit_transform(texts)
        print(f"TF-IDF matrix shape: {tfidf_matrix.shape}")
        
        # Понижаем размерность с помощью SVD
        n_components = min(self.config.TFIDF_SVD_COMPONENTS, tfidf_matrix.shape[1])
        self.svd = TruncatedSVD(
            n_components=n_components,
            random_state=self.config.RANDOM_STATE
        )
        
        reduced_features = self.svd.fit_transform(tfidf_matrix)
        print(f"Reduced features shape: {reduced_features.shape}")
        print(f"Explained variance ratio: {self.svd.explained_variance_ratio_.sum():.3f}")
        
        return reduced_features
    
    def transform(self, texts: List[str]) -> np.ndarray:
        """
        Преобразует новые тексты в признаки
        
        Args:
            texts: Список текстов
            
        Returns:
            Матрица TF-IDF признаков с пониженной размерностью
        """
        if self.tfidf_vectorizer is None or self.svd is None:
            raise ValueError("Сначала нужно обучить модель с помощью fit_transform()")
        
        texts = [str(text) if pd.notna(text) else '' for text in texts]
        tfidf_matrix = self.tfidf_vectorizer.transform(texts)
        return self.svd.transform(tfidf_matrix)
    
    def transform(self, texts: List[str]) -> np.ndarray:
        """
        Преобразует новые тексты в признаки
        
        Args:
            texts: Список текстов
            
        Returns:
            Матрица TF-IDF признаков с пониженной размерностью
        """
        if self.tfidf_vectorizer is None or self.svd is None:
            raise ValueError("Сначала нужно обучить модель с помощью fit_transform()")
        
        texts = [str(text) if pd.notna(text) else '' for text in texts]
        tfidf_matrix = self.tfidf_vectorizer.transform(texts)
        return self.svd.transform(tfidf_matrix)


class FeatureEngineer:
    """Класс для генерации признаков"""
    
    def __init__(self, config: Config):
        self.config = config
        self.constants = Constants()
        self.tfidf_extractor = TFIDFFeatureExtractor(config)
        self.tfidf_features_columns = None
        
    def create_genre_features(self, books_meta: pd.DataFrame, 
                            genres_df: pd.DataFrame, 
                            book_genres_df: pd.DataFrame) -> pd.DataFrame:
        """
        Создает признаки на основе жанров книг
        
        Args:
            books_meta: Метаданные книг
            genres_df: Справочник жанров
            book_genres_df: Связь книг и жанров
            
        Returns:
            DataFrame с признаками жанров
        """
        if genres_df.empty or book_genres_df.empty:
            print("Нет данных о жанрах, пропускаем создание жанровых признаков")
            return books_meta.copy()
        
        print("Создание жанровых признаков...")
        
        # Объединяем жанры с книгами
        book_genres = book_genres_df.merge(
            genres_df[[self.constants.COL_GENRE_ID, self.constants.COL_GENRE_NAME]], 
            on=self.constants.COL_GENRE_ID
        )
        
        # Создаем one-hot encoding для топ-N жанров
        top_n = 20
        top_genres = genres_df.nlargest(top_n, 'books_count')[self.constants.COL_GENRE_ID].tolist()
        
        # Создаем бинарные признаки для каждого топ-жанра
        genre_features = pd.DataFrame(index=books_meta[self.constants.COL_BOOK_ID].unique())
        
        for genre_id in tqdm(top_genres, desc="Creating genre features"):
            books_with_genre = book_genres[
                book_genres[self.constants.COL_GENRE_ID] == genre_id
            ][self.constants.COL_BOOK_ID].unique()
            
            genre_features[f'genre_{genre_id}'] = 0
            genre_features.loc[genre_features.index.isin(books_with_genre), f'genre_{genre_id}'] = 1
        
        # Считаем количество жанров у книги
        genre_counts = book_genres.groupby(self.constants.COL_BOOK_ID)[self.constants.COL_GENRE_ID].count()
        genre_features['genre_count'] = genre_counts
        genre_features['genre_count'] = genre_features['genre_count'].fillna(0)
        
        # Сбрасываем индекс для слияния
        genre_features = genre_features.reset_index().rename(
            columns={'index': self.constants.COL_BOOK_ID}
        )
        
        # Объединяем с метаданными книг
        books_with_genres = books_meta.merge(genre_features, on=self.constants.COL_BOOK_ID, how='left')
        
        # Заполняем пропуски
        genre_columns = [col for col in genre_features.columns if col != self.constants.COL_BOOK_ID]
        books_with_genres[genre_columns] = books_with_genres[genre_columns].fillna(0)
        
        return books_with_genres
    
    def create_tfidf_features(self, books_meta: pd.DataFrame, 
                             descriptions_df: pd.DataFrame,
                             is_train: bool = True) -> Tuple[pd.DataFrame, List[str]]:
        """
        Создает TF-IDF признаки из описаний книг
        
        Args:
            books_meta: Метаданные книг
            descriptions_df: Описания книг
            is_train: Если True - обучает TF-IDF, если False - только трансформирует
            
        Returns:
            Tuple[DataFrame с признаками, список имен TF-IDF колонок]
        """
        if descriptions_df.empty:
            print("Нет данных об описаниях, пропускаем TF-IDF")
            return books_meta.copy(), []
        
        print("Создание TF-IDF признаков...")
        
        # Объединяем описания с книгами
        books_with_descriptions = books_meta.merge(
            descriptions_df, on=self.constants.COL_BOOK_ID, how='left'
        )
        
        # Заполняем пропуски
        books_with_descriptions[self.constants.COL_DESCRIPTION] = books_with_descriptions[
            self.constants.COL_DESCRIPTION
        ].fillna('')
        
        # Создаем комбинированный текст (название + описание)
        books_with_descriptions['combined_text'] = (
            books_with_descriptions[self.constants.COL_TITLE].fillna('') + ' ' +
            books_with_descriptions[self.constants.COL_DESCRIPTION].fillna('')
        )
        
        # Обучаем или трансформируем TF-IDF
        combined_texts = books_with_descriptions['combined_text'].tolist()
        
        if is_train:
            # Для обучения используем fit_transform
            tfidf_features_array = self.tfidf_extractor.fit_transform(combined_texts)
        else:
            # Для валидации/теста используем transform
            if self.tfidf_extractor.tfidf_vectorizer is None:
                # Если модель не обучена, создаем пустые признаки
                print("Предупреждение: TF-IDF модель не обучена, создаем пустые признаки")
                n_components = self.config.TFIDF_SVD_COMPONENTS
                tfidf_features_array = np.zeros((len(books_with_descriptions), n_components))
            else:
                tfidf_features_array = self.tfidf_extractor.transform(combined_texts)
        
        # Создаем имена колонок для TF-IDF признаков
        n_features = tfidf_features_array.shape[1]
        tfidf_cols = [f'tfidf_{i}' for i in range(n_features)]
        
        # Преобразуем в DataFrame
        tfidf_df = pd.DataFrame(tfidf_features_array, columns=tfidf_cols)
        
        # Объединяем с исходными данными
        result_df = pd.concat([books_with_descriptions, tfidf_df], axis=1)
        
        # Удаляем временные текстовые колонки
        result_df = result_df.drop(['combined_text'], axis=1, errors='ignore')
        
        return result_df, tfidf_cols
    
    def create_temporal_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Создает временные признаки из timestamp
        
        Args:
            df: DataFrame с timestamp
            
        Returns:
            DataFrame с добавленными временными признаками
        """
        data = df.copy()
        
        if self.constants.COL_TIMESTAMP in data.columns:
            data['timestamp_dt'] = pd.to_datetime(data[self.constants.COL_TIMESTAMP])
            data['year'] = data['timestamp_dt'].dt.year
            data['month'] = data['timestamp_dt'].dt.month
            data['day'] = data['timestamp_dt'].dt.day
            data['dayofweek'] = data['timestamp_dt'].dt.dayofweek
            data['hour'] = data['timestamp_dt'].dt.hour
            
            # Удаляем временную колонку
            data = data.drop(columns=['timestamp_dt'])
        
        return data
    
    def create_user_book_features(self, df: pd.DataFrame, 
                                 train_interactions: pd.DataFrame) -> pd.DataFrame:
        """
        Создает признаки на основе взаимодействий пользователей и книг
        
        Args:
            df: DataFrame для добавления признаков
            train_interactions: DataFrame с тренировочными взаимодействиями
            
        Returns:
            DataFrame с добавленными признаками
        """
        data = df.copy()
        
        # --- Статистики пользователей ---
        print("Создание пользовательских статистик...")
        user_stats = train_interactions.groupby(self.constants.COL_USER_ID).agg({
            self.constants.COL_RATING: ['mean', 'std', 'count', 'max', 'min'],
            self.constants.COL_HAS_READ: ['sum', 'mean', 'std'],
            self.constants.COL_TIMESTAMP: ['max', 'min', 'nunique'] if self.constants.COL_TIMESTAMP in train_interactions.columns else []
        })
        
        # Выравниваем мультииндекс
        user_stats.columns = ['_'.join(col).strip() for col in user_stats.columns.values]
        user_stats = user_stats.reset_index()
        
        # Переименовываем колонки
        rename_dict = {
            'rating_mean': 'user_avg_rating',
            'rating_std': 'user_rating_std',
            'rating_count': 'user_activity_count',
            'rating_max': 'user_max_rating',
            'rating_min': 'user_min_rating',
            'has_read_sum': 'user_read_count',
            'has_read_mean': 'user_read_ratio',
            'has_read_std': 'user_read_std',
            'timestamp_max': 'user_last_activity',
            'timestamp_min': 'user_first_activity',
            'timestamp_nunique': 'user_active_days'
        }
        
        user_stats = user_stats.rename(columns={k: v for k, v in rename_dict.items() if k in user_stats.columns})
        
        # --- Статистики книг ---
        print("Создание книжных статистик...")
        book_stats = train_interactions.groupby(self.constants.COL_BOOK_ID).agg({
            self.constants.COL_RATING: ['mean', 'std', 'count', 'max', 'min'],
            self.constants.COL_HAS_READ: ['sum', 'mean', 'std'],
            self.constants.COL_USER_ID: 'nunique'
        })
        
        book_stats.columns = ['_'.join(col).strip() for col in book_stats.columns.values]
        book_stats = book_stats.reset_index()
        
        rename_dict = {
            'rating_mean': 'book_avg_rating',
            'rating_std': 'book_rating_std',
            'rating_count': 'book_popularity',
            'rating_max': 'book_max_rating',
            'rating_min': 'book_min_rating',
            'has_read_sum': 'book_read_count',
            'has_read_mean': 'book_read_ratio',
            'has_read_std': 'book_read_std',
            'user_id_nunique': 'book_unique_users'
        }
        
        book_stats = book_stats.rename(columns={k: v for k, v in rename_dict.items() if k in book_stats.columns})
        
        # Объединяем статистики
        data = data.merge(user_stats, on=self.constants.COL_USER_ID, how='left')
        data = data.merge(book_stats, on=self.constants.COL_BOOK_ID, how='left')
        
        return data
    
    def create_advanced_features(self, data: pd.DataFrame, 
                                train_interactions: pd.DataFrame) -> pd.DataFrame:
        """
        Создает продвинутые признаки
        
        Args:
            data: DataFrame с базовыми признаками
            train_interactions: Тренировочные взаимодействия
            
        Returns:
            DataFrame с добавленными продвинутыми признаками
        """
        df = data.copy()
        
        # --- Взаимодействия между пользователями и авторами ---
        print("Создание признаков пользователь-автор...")
        
        # Количество взаимодействий пользователя с автором
        user_author_stats = train_interactions.merge(
            df[[self.constants.COL_BOOK_ID, self.constants.COL_AUTHOR_ID]].drop_duplicates(),
            on=self.constants.COL_BOOK_ID,
            how='left'
        )
        
        if not user_author_stats.empty:
            user_author_agg = user_author_stats.groupby(
                [self.constants.COL_USER_ID, self.constants.COL_AUTHOR_ID]
            ).agg({
                self.constants.COL_RATING: ['mean', 'count'],
                self.constants.COL_HAS_READ: 'sum'
            })
            
            user_author_agg.columns = ['_'.join(col).strip() for col in user_author_agg.columns.values]
            user_author_agg = user_author_agg.reset_index()
            user_author_agg = user_author_agg.rename(columns={
                'rating_mean': 'user_author_avg_rating',
                'rating_count': 'user_author_interactions',
                'has_read_sum': 'user_author_read_count'
            })
            
            df = df.merge(
                user_author_agg, 
                on=[self.constants.COL_USER_ID, self.constants.COL_AUTHOR_ID], 
                how='left'
            )
        
        # --- Разница между средним рейтингом пользователя и книги ---
        if 'user_avg_rating' in df.columns and 'book_avg_rating' in df.columns:
            df['rating_diff'] = df['user_avg_rating'] - df['book_avg_rating']
            df['rating_diff_abs'] = df['rating_diff'].abs()
        
        # --- Время с последней активности пользователя (если есть timestamp) ---
        if self.constants.COL_TIMESTAMP in train_interactions.columns:
            latest_timestamp = train_interactions[self.constants.COL_TIMESTAMP].max()
            train_interactions['days_since_last'] = (
                latest_timestamp - train_interactions[self.constants.COL_TIMESTAMP]
            ).dt.days
            
            user_recency = train_interactions.groupby(self.constants.COL_USER_ID)['days_since_last'].min()
            df = df.merge(user_recency.rename('user_days_since_last'), 
                         on=self.constants.COL_USER_ID, how='left')
        
        return df
    
    def generate_features(self, df: pd.DataFrame, users_meta: pd.DataFrame,
                         books_meta: pd.DataFrame, train_interactions: pd.DataFrame,
                         val_interactions: pd.DataFrame = None,  # Добавляем val_interactions
                         genres_df: pd.DataFrame = None, 
                         book_genres_df: pd.DataFrame = None,
                         descriptions_df: pd.DataFrame = None,
                         is_train: bool = True) -> pd.DataFrame:  # Флаг - тренировочные ли данные
        """
        Основной метод для генерации всех признаков
        
        Args:
            df: Исходный DataFrame
            users_meta: Метаданные пользователей
            books_meta: Метаданные книг
            train_interactions: Тренировочные взаимодействия (ТОЛЬКО для train)
            val_interactions: Валидационные взаимодействия (для validation)
            genres_df: Справочник жанров
            book_genres_df: Связь книг и жанров
            descriptions_df: Описания книг
            is_train: Флаг - генерируем признаки для train или val/test
        """
        print("Генерация признаков...")
        
        # Копируем данные
        data = df.copy()
        
        # 1. Добавляем метаданные пользователей
        print("1. Добавление метаданных пользователей...")
        data = data.merge(users_meta, on=self.constants.COL_USER_ID, how='left')
        
        # 2. Обрабатываем метаданные книг с TF-IDF и жанрами
        print("2. Обработка метаданных книг...")
        processed_books = books_meta.copy()
        
        # Добавляем жанровые признаки если есть данные
        if genres_df is not None and book_genres_df is not None:
            processed_books = self.create_genre_features(processed_books, genres_df, book_genres_df)
        
        # Добавляем TF-IDF признаки если есть описания
        if descriptions_df is not None and not descriptions_df.empty:
            processed_books, tfidf_cols = self.create_tfidf_features(processed_books, descriptions_df)
        else:
            tfidf_cols = []
        
        # Объединяем обработанные метаданные книг
        data = data.merge(processed_books, on=self.constants.COL_BOOK_ID, how='left')
        
        # 3. КРИТИЧЕСКОЕ ИЗМЕНЕНИЕ: Разные данные для вычисления статистик
        print("3. Создание признаков взаимодействий...")
        
        if is_train:
            # Для тренировочных данных используем только train_interactions
            stats_data = train_interactions
        else:
            # Для валидационных/тестовых данных используем только train_interactions
            # НЕ используем val_interactions для вычисления статистик!
            stats_data = train_interactions
        
        data = self.create_user_book_features(data, stats_data)
        
        # 4. Создаем продвинутые признаки
        print("4. Создание продвинутых признаков...")
        data = self.create_advanced_features(data, stats_data)  # Используем stats_data здесь тоже
        
        # 5. Создаем временные признаки если есть timestamp
        if self.constants.COL_TIMESTAMP in data.columns:
            print("5. Создание временных признаков...")
            data = self.create_temporal_features(data)
        
        
        # 6. Обработка пропусков
        print("6. Обработка пропусков...")
        
        # Заполняем числовые признаки
        numeric_cols = data.select_dtypes(include=[np.number]).columns
        numeric_cols = [col for col in numeric_cols if col not in [
            self.constants.COL_USER_ID, self.constants.COL_BOOK_ID, self.constants.COL_TARGET
        ]]
        
        for col in numeric_cols:
            if col in data.columns:
                if 'user_' in col:
                    # Для пользовательских признаков заполняем глобальным средним или 0
                    if col.endswith('_ratio') or col.endswith('_mean'):
                        fill_value = train_interactions[self.constants.COL_RATING].mean() if 'rating' in col else 0
                    else:
                        fill_value = 0
                elif 'book_' in col:
                    # Для книжных признаков
                    if col.endswith('_rating'):
                        fill_value = train_interactions[self.constants.COL_RATING].mean()
                    else:
                        fill_value = 0
                else:
                    fill_value = data[col].median() if data[col].notna().any() else 0
                
                data[col] = data[col].fillna(fill_value)
        
        # 7. Обрабатываем категориальные признаки
        print("7. Обработка категориальных признаков...")
        for cat_feature in self.config.CATEGORICAL_FEATURES:
            if cat_feature in data.columns:
                # Для категориальных признаков заполняем -1 и преобразуем в int
                data[cat_feature] = data[cat_feature].fillna(-1).astype(int)
        
        # 8. Удаляем текстовые колонки (CatBoost не может обработать их как числовые)
        print("8. Удаление текстовых колонок...")
        text_cols_to_drop = [col for col in self.config.TEXT_FEATURES_TO_DROP if col in data.columns]
        if text_cols_to_drop:
            print(f"Удаляемые текстовые колонки: {text_cols_to_drop}")
            data = data.drop(columns=text_cols_to_drop)
        
        # Также удаляем любые другие нечисловые колонки
        non_numeric_cols = data.select_dtypes(exclude=[np.number]).columns
        non_numeric_cols = [col for col in non_numeric_cols if col not in [
            self.constants.COL_USER_ID, self.constants.COL_BOOK_ID, self.constants.COL_TARGET
        ]]
        
        if non_numeric_cols:
            print(f"Удаляемые нечисловые колонки: {non_numeric_cols}")
            data = data.drop(columns=non_numeric_cols)
        
        print(f"Признаки сгенерированы. Итоговая форма: {data.shape}")
        print(f"Количество признаков: {len(data.columns) - 3}")  # -3 для user_id, book_id, target
        
        return data
    
    def process_features(self, df: pd.DataFrame,
                        users_meta: pd.DataFrame,
                        books_meta: pd.DataFrame,
                        train_interactions: pd.DataFrame) -> pd.DataFrame:
        """
        Обертка для generate_features (для совместимости)
        """
        return self.generate_features(df, users_meta, books_meta, train_interactions)


# ============ ИСПРАВЛЕННЫЙ SubmissionGenerator ============

class SubmissionGenerator:
    """Класс для формирования финального сабмита"""
    
    def __init__(self, config: Config):
        self.config = config
        self.constants = Constants()
    
    def prepare_submission_format(self, candidates_df: pd.DataFrame) -> pd.DataFrame:
        """
        Преобразует candidates.csv в формат для предсказания
        
        Args:
            candidates_df: DataFrame с кандидатами в формате user_id, book_id_list
            
        Returns:
            DataFrame в формате user_id, book_id
        """
        print("Подготовка данных для предсказания...")
        
        # Проверяем формат candidates_df
        if 'book_id_list' in candidates_df.columns:
            # Преобразуем строку с book_id_list в список book_id
            candidates_long = []
            
            for _, row in tqdm(candidates_df.iterrows(), total=len(candidates_df), desc="Processing candidates"):
                user_id = row['user_id']
                book_id_str = str(row['book_id_list'])
                
                # Разделяем строку по запятым и удаляем дубликаты
                book_ids = []
                seen = set()
                for bid in book_id_str.split(','):
                    bid_clean = bid.strip()
                    if bid_clean and bid_clean not in seen:
                        book_ids.append(int(bid_clean))
                        seen.add(bid_clean)
                
                for book_id in book_ids:
                    candidates_long.append([user_id, book_id])
            
            candidates_long_df = pd.DataFrame(candidates_long, columns=['user_id', 'book_id'])
        else:
            # Уже в длинном формате
            candidates_long_df = candidates_df.copy()
        
        # Удаляем дубликаты пар (user_id, book_id)
        candidates_long_df = candidates_long_df.drop_duplicates(subset=['user_id', 'book_id'])
        
        print(f"Преобразовано в длинный формат: {len(candidates_long_df)} уникальных пар (user_id, book_id)")
        return candidates_long_df
    
    def generate_submission(self, pipeline: 'RankingPipeline', candidates_df: pd.DataFrame,
                          users_df: pd.DataFrame, books_df: pd.DataFrame, 
                          train_df: pd.DataFrame, 
                          book_descriptions: pd.DataFrame = None,
                          genres_df: pd.DataFrame = None,
                          book_genres_df: pd.DataFrame = None) -> pd.DataFrame:
        """
        Генерирует финальный сабмит
        
        Args:
            pipeline: Обученный пайплайн
            candidates_df: DataFrame с кандидатами
            users_df: Метаданные пользователей
            books_df: Метаданные книг
            train_df: Обучающие данные для вычисления статистик
            book_descriptions: Описания книг
            genres_df: Справочник жанров
            book_genres_df: Связь книг и жанров
            
        Returns:
            DataFrame с сабмитом в правильном формате
        """
        print("=" * 50)
        print("ФОРМИРОВАНИЕ САБМИТА")
        print("=" * 50)
        
        # 1. Подготавливаем кандидатов в длинном формате
        print("\n1. Подготовка кандидатов...")
        candidates_long = self.prepare_submission_format(candidates_df)
        
        # 2. Генерируем признаки для кандидатов
        print("\n2. Генерация признаков для кандидатов...")
        
        # Создаем фиктивный target для кандидатов
        candidates_long['target'] = 0
        
        # Генерируем признаки
        X_candidates = pipeline.feature_engineer.generate_features(
            candidates_long,
            users_df,
            books_df,
            train_df,  # train_interactions
            None,      # val_interactions - передаем None, так как для сабмита нет валидационных данных
            genres_df,
            book_genres_df,
            book_descriptions,
            is_train=False  # Важно! Для сабмита это не тренировочные данные
        )
        
        # 3. Делаем предсказания
        print("\n3. Предсказание релевантности...")
        
        # Подготавливаем данные для предсказания
        features_to_drop = ['user_id', 'book_id', 'target']
        X_pred = X_candidates.drop(features_to_drop, axis=1)
        
        # Делаем предсказания
        predictions = pipeline.model.predict(X_pred)
        
        # Добавляем предсказания
        X_candidates['prediction'] = predictions
        
        # 4. Формируем сабмит в требуемом формате
        print("\n4. Формирование сабмита...")
        
        # Сортируем по user_id и prediction (по убыванию)
        X_candidates = X_candidates.sort_values(['user_id', 'prediction'], ascending=[True, False])
        
        # Группируем по user_id и собираем топ-20 book_id
        submission_list = []
        
        for user_id, group in tqdm(X_candidates.groupby('user_id'), desc="Forming submission"):
            # Удаляем дубликаты book_id в рамках одного пользователя
            group_unique = group.drop_duplicates(subset='book_id')
            
            # Берем топ-20 или меньше, если кандидатов меньше
            top_k = min(20, len(group_unique))
            top_books = group_unique.head(top_k)['book_id'].tolist()
            
            # Проверяем на дубликаты
            if len(top_books) != len(set(top_books)):
                # Удаляем дубликаты, сохраняя порядок
                seen = set()
                top_books_unique = []
                for book_id in top_books:
                    if book_id not in seen:
                        seen.add(book_id)
                        top_books_unique.append(book_id)
                top_books = top_books_unique[:20]  # Берем снова топ-20 после удаления дубликатов
            
            # Преобразуем список в строку, разделенную запятыми
            book_id_list = ','.join(map(str, top_books))
            submission_list.append([user_id, book_id_list])
        
        # Создаем DataFrame сабмита
        submission_df = pd.DataFrame(submission_list, columns=['user_id', 'book_id_list'])
        
        # 5. Проверяем сабмит на соответствие требованиям
        print("\n5. Проверка сабмита...")
        submission_df = self._validate_submission(submission_df)
        
        print(f"\nСабмит сформирован для {len(submission_df)} пользователей")
        print(f"Среднее количество книг на пользователя: {submission_df['book_id_list'].apply(lambda x: len(x.split(',')) if x else 0).mean():.1f}")
        
        return submission_df
    
    def _validate_submission(self, submission_df: pd.DataFrame):
        """
        Проверяет сабмит на соответствие требованиям
        
        Args:
            submission_df: DataFrame с сабмитом
            
        Returns:
            Проверенный DataFrame
        """
        errors = []
        
        for idx, row in submission_df.iterrows():
            user_id = row['user_id']
            book_id_list = row['book_id_list']
            
            # Проверяем, что book_id_list не пустой
            if not book_id_list:
                errors.append({
                    'row': idx,
                    'user_id': user_id,
                    'error': 'Empty book_id_list'
                })
                continue
            
            # Разбираем список book_id
            book_ids = [bid.strip() for bid in book_id_list.split(',')]
            
            # Проверяем, что все book_id - числа
            for bid in book_ids:
                if not bid.isdigit():
                    errors.append({
                        'row': idx,
                        'user_id': user_id,
                        'error': f'Non-numeric book_id: {bid}'
                    })
            
            # Проверяем на дубликаты
            if len(book_ids) != len(set(book_ids)):
                duplicates = set([bid for bid in book_ids if book_ids.count(bid) > 1])
                errors.append({
                    'row': idx,
                    'user_id': user_id,
                    'error': f'Duplicate book_ids in list: {duplicates}'
                })
            
            # Проверяем количество книг (не более 20)
            if len(book_ids) > 20:
                errors.append({
                    'row': idx,
                    'user_id': user_id,
                    'error': f'More than 20 books: {len(book_ids)}'
                })
        
        if errors:
            print(f"\nНайдено {len(errors)} ошибок в сабмите:")
            for error in errors[:5]:  # Показываем только первые 5 ошибок
                print(f"  Строка {error['row']}, user_id={error['user_id']}: {error['error']}")
            if len(errors) > 5:
                print(f"  ... и еще {len(errors) - 5} ошибок")
            
            # Автоматически исправляем ошибки
            print("\nАвтоматическое исправление ошибок...")
            submission_df = self._fix_submission_errors(submission_df, errors)
        
        else:
            print("✓ Сабмит прошел все проверки!")
        
        return submission_df
    
    def _fix_submission_errors(self, submission_df: pd.DataFrame, errors: List[Dict]) -> pd.DataFrame:
        """
        Автоматически исправляет ошибки в сабмите
        
        Args:
            submission_df: DataFrame с сабмитом
            errors: Список ошибок
            
        Returns:
            Исправленный DataFrame
        """
        submission_df_fixed = submission_df.copy()
        
        for error in errors:
            idx = error['row']
            user_id = error['user_id']
            
            # Исправляем дубликаты
            if 'Duplicate' in error['error']:
                book_id_list = submission_df_fixed.at[idx, 'book_id_list']
                if pd.isna(book_id_list):
                    continue
                
                # Удаляем дубликаты, сохраняя порядок
                book_ids = [bid.strip() for bid in book_id_list.split(',')]
                seen = set()
                unique_book_ids = []
                
                for bid in book_ids:
                    if bid not in seen:
                        seen.add(bid)
                        unique_book_ids.append(bid)
                
                # Ограничиваем 20 книгами
                unique_book_ids = unique_book_ids[:20]
                
                # Обновляем строку
                submission_df_fixed.at[idx, 'book_id_list'] = ','.join(unique_book_ids)
        
        print("✓ Автоматическое исправление завершено")
        
        # Проверяем снова
        print("\nПовторная проверка после исправления...")
        submission_df_fixed = self._validate_submission(submission_df_fixed)
        
        return submission_df_fixed
    
    def save_submission(self, submission_df: pd.DataFrame, filename: str = "submission.csv"):
        """
        Сохраняет сабмит в файл
        
        Args:
            submission_df: DataFrame с сабмитом
            filename: Имя файла для сохранения
        """
        submission_path = self.config.RESULTS_DIR / filename
        
        # Сохраняем без индекса
        submission_df.to_csv(submission_path, index=False)
        
        print(f"\nСабмит сохранен: {submission_path}")
        print(f"Размер файла: {submission_path.stat().st_size / 1024:.1f} KB")
        
        # Создаем пример для проверки
        sample_path = self.config.RESULTS_DIR / "submission_sample.txt"
        with open(sample_path, 'w') as f:
            f.write("Пример формата сабмита:\n")
            f.write("=" * 50 + "\n")
            f.write(submission_df.head(10).to_string())
            f.write("\n\nСтатистика:\n")
            f.write(f"Количество пользователей: {len(submission_df)}\n")
            
            book_counts = submission_df['book_id_list'].apply(lambda x: len(x.split(',')) if x else 0)
            f.write(f"Среднее количество книг на пользователя: {book_counts.mean():.1f}\n")
            f.write(f"Мин. количество книг: {book_counts.min()}\n")
            f.write(f"Макс. количество книг: {book_counts.max()}\n")
            
            # Проверка уникальности user_id
            if submission_df['user_id'].nunique() == len(submission_df):
                f.write("✓ Все user_id уникальны\n")
            else:
                f.write(f"✗ Есть дубликаты user_id: {len(submission_df) - submission_df['user_id'].nunique()}\n")
        
        print(f"Пример сохранен в: {sample_path}")


# ============ ОБНОВЛЕННЫЙ RankingPipeline ============

class RankingPipeline:
    """Основной пайплайн для ранжирования"""
    
    def __init__(self, config: Config):
        self.config = config
        self.constants = Constants()
        self.data_loader = DataLoader(config)
        self.feature_engineer = FeatureEngineer(config)
        self.submission_generator = SubmissionGenerator(config)
        self.model = None
        self.data = None
    
    def run(self, train_mode: bool = True):
        """Запуск полного пайплайна"""
        print("=" * 50)
        print("ЗАПУСК ПАЙПЛАЙНА РАНЖИРОВАНИЯ")
        print("=" * 50)
        
        # 1. Загрузка данных
        print("\n1. Загрузка данных...")
        self.data = self.data_loader.load_all_data()
        
        train_df = self.data['train']
        candidates_df = self.data['candidates']
        users_df = self.data['users']
        books_df = self.data['books']
        book_descriptions = self.data.get('book_descriptions', pd.DataFrame())
        genres_df = self.data.get('genres', pd.DataFrame())
        book_genres_df = self.data.get('book_genres', pd.DataFrame())
        
        # Подготовка целевой переменной
        train_df = self.data_loader.prepare_target(train_df)
        
        if train_mode:
            # 2. Разделение на train/val С УЧЕТОМ ВРЕМЕНИ
            print("\n2. Разделение данных с учетом времени...")
            
            # Сортируем по времени
            train_df = train_df.sort_values(self.constants.COL_TIMESTAMP)
            
            # Разделяем по времени (например, 90% времени - train, 10% - val)
            split_time = train_df[self.constants.COL_TIMESTAMP].quantile(0.9)
            
            train_interactions = train_df[train_df[self.constants.COL_TIMESTAMP] < split_time].copy()
            val_interactions = train_df[train_df[self.constants.COL_TIMESTAMP] >= split_time].copy()
            
            # Берем только пользователей, которые есть в train
            val_users_in_train = set(train_interactions[self.constants.COL_USER_ID].unique())
            val_interactions = val_interactions[
                val_interactions[self.constants.COL_USER_ID].isin(val_users_in_train)
            ]
            
            print(f"Train interactions: {len(train_interactions)} (до {split_time})")
            print(f"Val interactions: {len(val_interactions)} (после {split_time})")
            
            # 3. Генерация кандидатов с помощью ALS
            print("\n3. Генерация кандидатов с помощью ALS...")
            
            # Обучаем ALS на тренировочных взаимодействиях (ДО split_time)
            als_recommender = ALSRecommender(self.config)
            als_recommender.train(train_interactions)
            
            # Генерируем кандидатов для пользователей в валидации
            val_users = val_interactions[self.constants.COL_USER_ID].unique()
            n_candidates = self.config.ALS_TOP_K_CANDIDATES
            
            generated_candidates_val = als_recommender.generate_candidates(
                train_interactions,  # Только train для генерации кандидатов
                val_users,
                n_candidates=n_candidates
            )
            
            # Для тренировочных пользователей тоже генерируем кандидатов
            train_users = train_interactions[self.constants.COL_USER_ID].unique()
            generated_candidates_train = als_recommender.generate_candidates(
                train_interactions,
                train_users,
                n_candidates=n_candidates
            )
            
            # Объединяем кандидатов
            generated_candidates = pd.concat([
                generated_candidates_train,
                generated_candidates_val
            ], ignore_index=True)
            
            # 4. Размечаем кандидатов
            print("\n4. Разметка кандидатов...")
            
            # Создаем множества реальных взаимодействий для быстрой проверки
            real_interactions_train_set = set(
                zip(train_interactions[self.constants.COL_USER_ID], 
                    train_interactions[self.constants.COL_BOOK_ID])
            )
            
            real_interactions_val_set = set(
                zip(val_interactions[self.constants.COL_USER_ID], 
                    val_interactions[self.constants.COL_BOOK_ID])
            )
            
            def is_real_interaction(row):
                user_id = row[self.constants.COL_USER_ID]
                book_id = row[self.constants.COL_BOOK_ID]
                
                if user_id in train_users_set:
                    return (user_id, book_id) in real_interactions_train_set
                else:
                    return (user_id, book_id) in real_interactions_val_set
            
            train_users_set = set(train_users)
            generated_candidates['is_real'] = generated_candidates.apply(is_real_interaction, axis=1)
            
            # 5. Создаем финальный датасет с таргетами
            print("\n5. Создание финального датасета...")
            
            # Позитивные примеры из train
            train_positives = train_interactions[[
                self.constants.COL_USER_ID, 
                self.constants.COL_BOOK_ID, 
                self.constants.COL_TARGET
            ]].copy()
            
            # Позитивные примеры из val
            val_positives = val_interactions[[
                self.constants.COL_USER_ID, 
                self.constants.COL_BOOK_ID, 
                self.constants.COL_TARGET
            ]].copy()
            
            # Негативные примеры (кандидаты, которых нет в реальных взаимодействиях)
            negative_candidates = generated_candidates[~generated_candidates['is_real']].copy()
            negative_candidates[self.constants.COL_TARGET] = self.constants.TARGET_NEGATIVE
            
            # Объединяем
            full_dataset = pd.concat([
                train_positives,
                val_positives,
                negative_candidates[[self.constants.COL_USER_ID, 
                                     self.constants.COL_BOOK_ID, 
                                     self.constants.COL_TARGET]]
            ], ignore_index=True)
            
            print(f"\nИтоговый размер датасета: {len(full_dataset)}")
            print(f"  - Позитивных (train): {len(train_positives)}")
            print(f"  - Позитивных (val): {len(val_positives)}")
            print(f"  - Негативных: {len(negative_candidates)}")
            
            # 6. Генерация признаков для train и val раздельно
            print("\n6. Генерация признаков...")
            
            # Разделяем данные на train и val части
            train_mask = full_dataset[self.constants.COL_USER_ID].isin(train_users_set)
            val_mask = ~train_mask
            
            # Генерируем признаки для train
            print("  Генерация признаков для train...")
            X_train = self.feature_engineer.generate_features(
                full_dataset[train_mask],
                users_df,
                books_df,
                train_interactions,  # Только train для статистик
                val_interactions,    # Передаем, но не используем в is_train=True
                genres_df,
                book_genres_df,
                book_descriptions,
                is_train=True
            )
            
            # Генерируем признаки для val
            print("  Генерация признаков для val...")
            X_val = self.feature_engineer.generate_features(
                full_dataset[val_mask],
                users_df,
                books_df,
                train_interactions,  # Только train для статистик!
                val_interactions,
                genres_df,
                book_genres_df,
                book_descriptions,
                is_train=False  # Важно! Для val не используем val_interactions в статистиках
            )
            
            # Объединяем
            X_full = pd.concat([X_train, X_val], ignore_index=True)
            
            # Определяем списки признаков
            categorical_features = [c for c in self.config.CATEGORICAL_FEATURES if c in X_full.columns]
            
            print(f"\nКатегориальные признаки ({len(categorical_features)}): {categorical_features[:10]}...")
            
            # 7. Подготовка данных для CatBoostRanker
            print("\n7. Подготовка данных для обучения...")
            
            # Сортируем по user_id (требование CatBoostRanker)
            X_full = X_full.sort_values(by=self.constants.COL_USER_ID)
            
            # Разделяем на признаки и целевую переменную
            features_to_drop = [self.constants.COL_USER_ID, 
                               self.constants.COL_BOOK_ID, 
                               self.constants.COL_TARGET]
            
            X = X_full.drop(features_to_drop, axis=1)
            y = X_full[self.constants.COL_TARGET]
            group_id = X_full[self.constants.COL_USER_ID]
            
            # Разделяем на train и val по пользователям
            train_mask_full = X_full[self.constants.COL_USER_ID].isin(train_users)
            val_mask_full = X_full[self.constants.COL_USER_ID].isin(val_users)
            
            # Создаем Pool объекты
            train_pool = Pool(
                data=X[train_mask_full],
                label=y[train_mask_full],
                group_id=group_id[train_mask_full],
                cat_features=categorical_features
            )
            
            val_pool = Pool(
                data=X[val_mask_full],
                label=y[val_mask_full],
                group_id=group_id[val_mask_full],
                cat_features=categorical_features
            )
            
            print(f"Train pool size: {train_pool.shape}")
            print(f"Val pool size: {val_pool.shape}")
            
            # 8. Обучение модели
            print("\n8. Обучение CatBoostRanker...")
            
            self.model = CatBoostRanker(**self.config.get_cb_params())
            self.model.fit(train_pool, eval_set=val_pool)
            
            print("Обучение завершено!")
            
            # 9. Сохранение модели
            print("\n9. Сохранение модели...")
            model_path = self.config.MODEL_DIR / "catboost_ranker.cbm"
            self.model.save_model(str(model_path))
            print(f"Модель сохранена: {model_path}")
            
            return self.model
        
        else:
            print("\nРежим предсказания: пропускаем обучение...")
            
            # Загружаем сохраненную модель
            model_path = self.config.MODEL_DIR / "catboost_ranker.cbm"
            if model_path.exists():
                print(f"Загрузка модели из {model_path}...")
                self.model = CatBoostRanker()
                self.model.load_model(str(model_path))
                print("Модель загружена!")
            else:
                raise FileNotFoundError(f"Модель не найдена: {model_path}")
        
        return self.model
    
    def generate_submission(self):
        """Генерирует финальный сабмит"""
        if self.data is None:
            raise ValueError("Данные не загружены. Сначала запустите run()")
        
        if self.model is None:
            raise ValueError("Модель не обучена. Сначала запустите run(train_mode=True)")
        
        # Генерируем сабмит
        submission_df = self.submission_generator.generate_submission(
            self,
            self.data['candidates'],
            self.data['users'],
            self.data['books'],
            self.data['train'],
            self.data.get('book_descriptions', pd.DataFrame()),
            self.data.get('genres', pd.DataFrame()),
            self.data.get('book_genres', pd.DataFrame())
        )
        
        # Сохраняем сабмит
        self.submission_generator.save_submission(submission_df)
        
        return submission_df


# ============ АЛЬТЕРНАТИВНЫЙ ПРОСТОЙ МЕТОД ============

def create_simple_submission(candidates_file: str, output_file: str = "submission.csv"):
    """
    Создает простой сабмит на основе исходных кандидатов
    (если модель не работает, можно использовать этот метод для создания базового сабмита)
    
    Args:
        candidates_file: Путь к файлу candidates.csv
        output_file: Имя выходного файла
    """
    print("Создание простого сабмита...")
    
    # Загружаем candidates
    candidates = pd.read_csv(candidates_file)
    
    submission_list = []
    
    for _, row in candidates.iterrows():
        user_id = row['user_id']
        book_id_list = row['book_id_list']
        
        # Разделяем строку на book_id
        book_ids = [bid.strip() for bid in str(book_id_list).split(',')]
        
        # Удаляем дубликаты
        seen = set()
        unique_book_ids = []
        for bid in book_ids:
            if bid and bid not in seen:
                unique_book_ids.append(bid)
                seen.add(bid)
        
        # Берем топ-20 или меньше
        top_k = min(20, len(unique_book_ids))
        top_books = unique_book_ids[:top_k]
        
        # Формируем строку
        book_id_str = ','.join(top_books)
        submission_list.append([user_id, book_id_str])
    
    # Создаем DataFrame
    submission_df = pd.DataFrame(submission_list, columns=['user_id', 'book_id_list'])
    
    # Сохраняем
    submission_df.to_csv(output_file, index=False)
    print(f"Сабмит сохранен: {output_file}")
    print(f"Количество пользователей: {len(submission_df)}")
    
    return submission_df


# ============ ПОЛНЫЙ ПАЙПЛАЙН С САБМИТОМ ============

def full_pipeline():
    """Полный пайплайн: обучение + создание сабмита"""
    # Инициализация
    seed_everything(Config.RANDOM_STATE)
    
    # Создание пайплайна
    pipeline = RankingPipeline(Config())
    
    try:
        # 1. Обучение модели
        print("\n" + "="*50)
        print("ЭТАП 1: ОБУЧЕНИЕ МОДЕЛИ")
        print("="*50)
        model = pipeline.run(train_mode=True)
        
        # 2. Создание сабмита
        print("\n" + "="*50)
        print("ЭТАП 2: ФОРМИРОВАНИЕ САБМИТА")
        print("="*50)
        submission = pipeline.generate_submission()
        
        # 3. Проверка формата сабмита
        print("\n" + "="*50)
        print("ЭТАП 3: ПРОВЕРКА ФОРМАТА САБМИТА")
        print("="*50)
        
        # Пример вывода первых строк сабмита
        print("\nПервые 5 строк сабмита:")
        print(submission.head())
        
        # Проверка количества книг на пользователя
        book_counts = submission['book_id_list'].apply(lambda x: len(x.split(',')) if x else 0)
        print(f"\nСтатистика по количеству книг на пользователя:")
        print(f"  - Минимум: {book_counts.min()}")
        print(f"  - Максимум: {book_counts.max()}")
        print(f"  - Среднее: {book_counts.mean():.1f}")
        print(f"  - Медиана: {book_counts.median():.1f}")
        
        # Проверка уникальности user_id
        if submission['user_id'].nunique() == len(submission):
            print(f"\n✓ Все user_id уникальны ({len(submission)} пользователей)")
        else:
            print(f"\n✗ Есть дубликаты user_id")
        
        print("\n" + "="*50)
        print("ПАЙПЛАЙН УСПЕШНО ЗАВЕРШЕН!")
        print("="*50)
        
        return model, submission
        
    except Exception as e:
        print(f"Ошибка при выполнении пайплайна: {e}")
        import traceback
        traceback.print_exc()
        return None, None


# ============ АЛЬТЕРНАТИВНЫЙ ПАЙПЛАЙН (ТОЛЬКО САБМИТ) ============

def submission_only_pipeline():
    """Пайплайн только для создания сабмита (если модель уже обучена)"""
    # Инициализация
    seed_everything(Config.RANDOM_STATE)
    
    # Создание пайплайна
    pipeline = RankingPipeline(Config())
    
    try:
        # 1. Загрузка данных
        print("\n" + "="*50)
        print("ЭТАП 1: ЗАГРУЗКА ДАННЫХ")
        print("="*50)
        pipeline.data = pipeline.data_loader.load_all_data()
        
        # 2. Загрузка модели
        print("\n" + "="*50)
        print("ЭТАП 2: ЗАГРУЗКА МОДЕЛИ")
        print("="*50)
        
        model_path = Config.MODEL_DIR / "catboost_ranker.cbm"
        if model_path.exists():
            print(f"Загрузка модели из {model_path}...")
            pipeline.model = CatBoostRanker()
            pipeline.model.load_model(str(model_path))
            print("Модель загружена!")
        else:
            print(f"Модель не найдена: {model_path}")
            print("Запускаем обучение модели...")
            pipeline.run(train_mode=True)
        
        # 3. Создание сабмита
        print("\n" + "="*50)
        print("ЭТАП 3: ФОРМИРОВАНИЕ САБМИТА")
        print("="*50)
        submission = pipeline.generate_submission()
        
        return submission
        
    except Exception as e:
        print(f"Ошибка при выполнении пайплайна: {e}")
        import traceback
        traceback.print_exc()
        return None


# ============ ТОЧКА ВХОДА ============

if __name__ == "__main__":
    # Вариант 1: Полный пайплайн (обучение + сабмит)
    print("Выберите режим работы:")
    print("1. Полный пайплайн (обучение + сабмит)")
    print("2. Только создание сабмита (если модель уже обучена)")
    print("3. Создать простой сабмит из candidates.csv (без модели)")
    
    # try:
    #     choice = int(input("Введите номер (1, 2 или 3): "))
    # except:
    #     choice = 1
    choice = 1
    
    if choice == 1:
        model, submission = full_pipeline()
    elif choice == 2:
        submission = submission_only_pipeline()
    elif choice == 3:
        # Простой сабмит без модели
        candidates_file = Config.DATA_DIR / "candidates.csv"
        if not candidates_file.exists():
            print(f"Файл не найден: {candidates_file}")
            # Попробуем найти в другом месте
            candidates_file = Path("/kaggle/input/nto-team-tour/public/candidates.csv")
        
        submission = create_simple_submission(
            candidates_file=str(candidates_file),
            output_file=str(Config.RESULTS_DIR / "simple_submission.csv")
        )
    else:
        print("Неверный выбор, запускаем полный пайплайн...")
        model, submission = full_pipeline()
    
    if submission is not None:
        print("\n" + "="*50)
        print("САБМИТ ГОТОВ!")
        print("="*50)
        print(f"Файл: {Config.RESULTS_DIR / 'submission.csv'}")
        print("\nПример формата сабмита:")
        print(submission.head(3).to_string())
        
        # Сохраняем также в формате для проверки
        sample_path = Config.RESULTS_DIR / "submission_sample.txt"
        with open(sample_path, 'w') as f:
            f.write(submission.head(10).to_string())
        print(f"\nПример сохранен в: {sample_path}")